Manipulators are verbs

In [ ]:
class Color(str: String = "")

class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [1]:
class Document {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }
}

defined class Document

The example above shows a Document class with a write method that serves as a manipulator. Following the principle, it has a verb name and returns Unit (void).

However, what if we need to know how many bytes were saved? Let's refactor according to the principles:


In [1]:
// Problem: This violates our principle - a manipulator returning a value
class DocumentProblem {
  def write(content: java.io.InputStream): Int = {
    // Implementation that writes and returns bytes written
    42 // Dummy return for example
  }
}

// Solution: Separate the concerns
class Document {
  def output(): OutputPipe = {
    new OutputPipe()
  }
}

class OutputPipe {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }

  def bytes: Int = {
    // Return number of bytes written
    42 // Dummy return for example
  }

  def time: Long = {
    // Return time taken for writing
    123L // Dummy return for example
  }
}

// Usage example
val doc = new Document()
val pipe = doc.output()
pipe.write(new java.io.ByteArrayInputStream("Hello".getBytes))
val bytesWritten = pipe.bytes
val timeElapsed = pipe.time

defined class DocumentProblem
defined class Document
defined class OutputPipe
doc: Document = ammonite.$sess.cmd1$Helper$Document@58e67ec3
pipe: OutputPipe = ammonite.$sess.cmd1$Helper$OutputPipe@3a133558
bytesWritten: Int = 42
timeElapsed: Long = 123L

Notice how we've refactored to maintain the principle:
1. `output()` is a builder - it returns a new object and has a noun name
2. `write()` is a manipulator - it has a verb name and returns Unit
3. `bytes` and `time` are builders - they have noun names and return values


The Builder Pattern and its concerns


In [ ]:
// Builder Pattern example (though the author recommends against it)
class Book private (
  val author: String = "",
  val title: String = "",
  val pages: List[String] = List.empty
) {
  def withAuthor(author: String): Book = {
    new Book(author, title, pages)
  }

  def withTitle(title: String): Book = {
    new Book(author, title, pages)
  }

  def withPage(page: String): Book = {
    new Book(author, title, pages :+ page)
  }
}

// Usage
val myBook = new Book()
  .withAuthor("John Doe")
  .withTitle("Clean Code")
  .withPage("Chapter 1")

The Builder Pattern methods follow the principle - they have noun-like names (with a prefix) and return values. However, as noted in the text, the author generally advises against this pattern as it often leads to less cohesive, larger objects.


A better approach to complex objects with many properties


In [ ]:
// Instead of a large Book class with a builder, break it into smaller objects
class Author(val name: String)
class Title(val text: String)
class Page(val content: String)

class Book(val author: Author, val title: Title, val pages: List[Page]) {
  def printInfo(): Unit = {
    println(s"${title.text} by ${author.name}, ${pages.size} pages")
  }
}

// Usage
val book = new Book(
  new Author("Jane Smith"),
  new Title("Elegant Objects"),
  List(new Page("Introduction"), new Page("Chapter 1"))
)
book.printInfo()

This approach creates smaller, more focused objects rather than using the Builder Pattern for complex initialization.


## Why This Principle Matters

This strict separation between builders and manipulators makes code:

1. **More readable** - Method names clearly indicate what they do
2. **More predictable** - Builders don't change state, manipulators don't return values
3. **More maintainable** - Single responsibility principle applied at the method level
4. **More testable** - Easier to test methods with clear, focused responsibilities


## Handling Complex Cases

Sometimes we need both information and manipulation. The solution is to create specific objects for each responsibility:


In [2]:
// Example: Managing a database connection
class Database(url: String) {
  // Builder - returns a connection object
  def connection: Connection = {
    new Connection(url)
  }

  // Builder - returns statistics object
  def stats: Statistics = {
    new Statistics(url)
  }
}

class Connection(url: String) {
  // Manipulator - executes a query
  def execute(sql: String): Unit = {
    println(s"Executing $sql on $url")
  }

  // Manipulator - closes the connection
  def close(): Unit = {
    println("Connection closed")
  }
}

class Statistics(url: String) {
  // Builder - returns query count
  def queryCount: Int = {
    // Implementation to get query count
    42
  }

  // Builder - returns connection time
  def connectionTime: Long = {
    // Implementation to get connection time
    1000L
  }
}

// Usage
val db = new Database("jdbc:postgresql://localhost:5432/mydb")
val conn = db.connection
conn.execute("SELECT * FROM users")
conn.close()

val stats = db.stats
println(s"Executed ${stats.queryCount} queries")
println(s"Connection time: ${stats.connectionTime}ms")


Executing SELECT * FROM users on jdbc:postgresql://localhost:5432/mydb
Connection closed
Executed 42 queries
Connection time: 1000ms


defined class Database
defined class Connection
defined class Statistics
db: Database = ammonite.$sess.cmd2$Helper$Database@87065b0
conn: Connection = ammonite.$sess.cmd2$Helper$Connection@13a93ecd
stats: Statistics = ammonite.$sess.cmd2$Helper$Statistics@6aacd07a

### Boolean Results

Boolean methods are an exception to the naming rules. They are builders (they return values), but instead of using nouns, they should use adjectives:


In [1]:
class File(path: String) {
  // Good: Using adjectives for Boolean methods
  def empty: Boolean = {
    // Implementation to check if file is empty
    true
  }

  def readable: Boolean = {
    // Implementation to check if file is readable
    true
  }

  def present: Boolean = {
    // Better than "exists" - "is present" sounds right
    true
  }

  // Bad examples (would be better as adjectives)
  def isEmpty: Boolean = empty // Not recommended - redundant "is" prefix
  def exists: Boolean = present // Not recommended - use "present" instead
}

case class User(name: String, age: Int) {
  // Good: "equal to" sounds right
  def equalTo(other: User): Boolean = {
    this.name == other.name && this.age == other.age
  }

  // Good: adjective for Boolean result
  def adult: Boolean = {
    age >= 18
  }
}

// Usage examples showing natural reading in conditionals
val file = new File("/tmp/data.txt")
val user = new User("John", 25)
val otherUser = new User("Jane", 30)

if (file.empty) {
  println("The file is empty") // Reads naturally: "if file is empty"
}

if (file.readable) {
  println("The file can be read") // Reads naturally: "if file is readable"
}

if (user.adult) {
  println("User is an adult") // Reads naturally: "if user is adult"
}

if (user.equalTo(otherUser)) {
  println("Users are the same") // Reads naturally: "if user is equal to other user"
}


The file is empty
The file can be read
User is an adult


defined class File
defined class User
file: File = ammonite.$sess.cmd1$Helper$File@782f660d
user: User = User(name = "John", age = 25)
otherUser: User = User(name = "Jane", age = 30)

## Summary of Boolean Method Naming Rules

1. **Boolean methods are builders** (they return values)
2. **Use adjectives for Boolean methods**, not nouns or verbs
3. **Don't use the "is" prefix** in the method name
4. **Test readability** by mentally placing "is" before the name - it should sound correct
5. **Examples of good names**:
   - `empty()` instead of `isEmpty()`
   - `readable()` instead of `canRead()`
   - `negative()` instead of `isNegative()`
   - `equalTo(obj)` instead of `equals(obj)`
   - `present()` instead of `exists()`

This naming convention makes conditionals read more naturally, as the Boolean methods essentially answer the question "is this object [adjective]?"

## 2.5 Don't use public constants

Public constants create global coupling and reduce cohesion.
Prefer keeping values private to the class that uses them, or pass them in as small objects (dependencies) instead of sharing a global constant.

In [1]:
// Bad example: global constant-like object (avoid)
object ConstantsBad {
  val CrLf: String = "\r\n"
}

class RowsBad2(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row => p.printf("{%s}%s", row, ConstantsBad.CrLf) }
  }
}

defined object ConstantsBad
defined class RowsBad2

Better alternatives:

- Keep the value private and local to the class.
- Or encapsulate the value as an object and inject it.

In [ ]:
class Rows2(private val all: List[String]) {
  private val CrLf: String = "\r\n"
  def print(p: java.io.PrintStream): Unit =
    all.foreach { row => p.printf("{%s}%s", row, CrLf) }
}

trait LineSeparator { def value: String }
class CrLf extends LineSeparator { val value = "\r\n" }
class Lf extends LineSeparator { val value = "\n" }

class Records2(private val all: List[String], private val sep: LineSeparator) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(sep.value)
    }
  }
}

## 2.5.1 Introduction of coupling

Public constants introduce hidden, hard-coded dependencies. When multiple classes reach out to a shared global value, they become tightly coupled to it and to each other, making behavior changes unpredictable.

Consider these two classes that both rely on a shared line separator constant:


In [ ]:
// Bad: shared global constant couples unrelated classes to the same value
object ConstantsGlobal {
  val CrLf: String = "\r\n"
}

class RecordsCoupled(private val all: List[String]) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(ConstantsGlobal.CrLf) // hard dependency here
    }
  }
}

class RowsCoupled(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row =>
      p.printf("{%s}%s", row, ConstantsGlobal.CrLf) // and here
    }
  }
}

Now, RecordsCoupled.write, RowsCoupled.print, and ConstantsGlobal.CrLf are all coupled. If we change CrLf (for example, to a platform-specific or protocol-specific value), the behavior of both classes changes, and it’s unclear whether those changes are acceptable. One user may be printing to the console, while another may be composing an HTTP message where the line ending is mandated and must not change.

The constant sits in a global scope without semantic context. We can’t tell how it’s used or which usages are safe to alter. This erodes maintainability: many objects depend on a value whose meaning varies by context.

A better design is to inject small objects that carry the context (as shown above with LineSeparator and Records2/Rows2), so each class depends on what it needs, and changes are localized.
